In [ ]:
from importlib import reload ,import_module
import module.utilize as utilize
import module.singleVariant as singleVariant
import module.multiVariant as multiVariant
import numpy as np
from numba import njit,jit, float32
from tqdm import tqdm
import module.multiHistogramSparse as multiHistogramSparse
import module.ananlysisFuncion as analysisFunction
import module.singleVariantCopulaBase as CopulaBase
import math
reload(multiHistogramSparse)
reload(utilize)
reload(multiVariant)
reload(singleVariant)
reload(analysisFunction)
reload(CopulaBase)

#attribute_names=np.array(["SALT","TEMP","U","V","W"])
attribute_names=np.array(["SALT","TEMP","U"])
all_ensamble_data=np.empty([0,180,20,250,250])
incremental_number=180

for name in attribute_names:
    data=utilize.readRedSeaFile(name)
    data=data.reshape(1,180,20,250,250)
    all_ensamble_data=np.append(all_ensamble_data,data,axis=0)





covBlockSize=5
dataBlockSize=5
binsNumber=128
sizeZ=20
sizeY=250
sizeX=250
minMaxBlockSize=2
isMinMax=False

conditions=np.array([[36,40],[26,30],[-math.inf,0]])

print("start fit model")
with tqdm(total=4, desc="Model fitting") as pbar:
    #oursModel=multiVariant.multiDistCopula3D(all_ensamble_data,dataBlockSize,covBlockSize,binsNumber,[sizeZ,sizeY,sizeX],minMaxBlockSize,isMinMax)
    oursModel=multiVariant.multiDistCopula3D.load(f"RedSea_{attribute_names.shape[0]}varaibles_{incremental_number}members_128Bins_dBlock5_cBlock5")
    #conditions=np.array([[0,1e5],[3e10,5e10]])
    #oursModel.fit()
    print("ours complete fit")
    pbar.update(1)
    copulaBlockSize=2
    copulaBaseModel=CopulaBase.multiVariantCopulaBase(all_ensamble_data,copulaBlockSize)
    copulaBaseModel.fit()
    print("copula complete fit")
    pbar.update(1)
    multiHistBlockSize=2
    multiHistModel=multiHistogramSparse.multiHistogramSpaseModel(all_ensamble_data,blockSize=multiHistBlockSize,binsNumber=binsNumber)
    multiHistModel.fit()
    print("multi-hist complete fit")
    pbar.update(1)
    gtModel=multiHistogramSparse.multiHistogramSpaseModel(all_ensamble_data,blockSize=1,binsNumber=binsNumber)
    gtModel.fit()

    multiBinEdges=gtModel.vBinEdges

    print("complete fit")
    pbar.update(1)



result=np.zeros([sizeZ,sizeY,sizeX],dtype=np.float32)
oursResult=np.zeros([sizeZ,sizeY,sizeX],dtype=np.float32)
copulaBaseResult=np.zeros([sizeZ,sizeY,sizeX],dtype=np.float32)
mtResult=np.zeros([sizeZ,sizeY,sizeX],dtype=np.float32)


#oursModel=multiVariant.multiDistCopula3D.load(f"RedSea_{attribute_names.shape[0]}varaibles_{incremental_number}members_128Bins_dBlock5_cBlock5")
with tqdm(total=sizeZ*sizeY*sizeX, desc="總進度") as pbar:
    for idx in range(sizeZ * sizeY * sizeX):
        
        z = idx // (sizeY * sizeX)
        y = (idx // sizeX) % sizeY
        x = idx % sizeX      

        #Ground Truth
        samples=gtModel.sampleByPos(z,y,x) 
        prob=analysisFunction.probability_in_range_numba(samples,conditions)
        result[z,y,x]=prob

        #Ours
        samples=oursModel.sampleByPos(z,y,x)
        prob=analysisFunction.probability_in_range_numba(samples,conditions)
        oursResult[z,y,x]=prob
        
        
        #Copula base
        samples=copulaBaseModel.sampleByPos(z,y,x)
        prob=analysisFunction.probability_in_range_numba(samples,conditions)
        copulaBaseResult[z,y,x]=prob

        samples=multiHistModel.sampleByPos(z,y,x)
        prob=analysisFunction.probability_in_range_numba(samples,conditions)
        mtResult[z,y,x]=prob

        pbar.update(1)
        


result.tofile(f"RedSeaGT_{attribute_names.shape[0]}varaibles.bin")
oursResult.tofile(f"RedSeaOus_{attribute_names.shape[0]}varaibles.bin")
copulaBaseResult.tofile(f"RedSeaCopulaBase_{attribute_names.shape[0]}varaibles.bin")
mtResult.tofile(f"RedSeaMtResult_{attribute_names.shape[0]}varaibles.bin")

start fit model


Model fitting:  25%|██▌       | 1/4 [00:00<00:00,  3.97it/s]

ours complete fit


Model fitting:  50%|█████     | 2/4 [04:40<05:29, 164.99s/it]

copula complete fit


Model fitting:  75%|███████▌  | 3/4 [09:24<03:39, 219.25s/it]

multi-hist complete fit


Model fitting: 100%|██████████| 4/4 [15:10<00:00, 227.52s/it]


complete fit


總進度: 100%|██████████| 1250000/1250000 [1:26:00<00:00, 242.25it/s]
